In [ ]:
import boto3
import json
import datetime
from botocore.exceptions import ClientError

bedrock_client = boto3.client('bedrock-runtime')
guardrail_client = boto3.client('bedrock')  # Assuming `bedrock` is the correct service for guardrails.

def lambda_handler(event, context):
    subject = "Meeting Update"
    email_body = "Please confirm your availability for the team meeting tomorrow."
    prompt = f"Generate a professional response for the following:\nSubject: {subject}\nEmail Body: {email_body}"

    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt}],
            }
        ],
    }

    request = json.dumps(native_request)

    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-5-sonnet-20240620-v1:0",
            body=request,
            contentType="application/json"
        )

        response_body = response["body"].read().decode("utf-8")
        model_response = json.loads(response_body)

        response_text = ""

        if "content" in model_response and len(model_response["content"]) > 0:
            response_text = model_response["content"][0].get("text", "No response generated")

        # Apply guardrail check
        guardrail_identifier = "custom-guardrail-id"  # Replace with your guardrail identifier
        guardrail_version = "1"  # Replace with the specific version

        guardrail_response = guardrail_client.get_guardrail(
            guardrailIdentifier=guardrail_identifier,
            version=guardrail_version,
            input=response_text
        )

        if guardrail_response.get("isBlocked", False):  # Assuming the response contains an 'isBlocked' field
            output_data = {
                "Date": datetime.date.today().isoformat(),
                "MessageID": context.aws_request_id if context else "test-message-id",
                "Body": "Sorry the model cannot generate the response for this request."
            }
        else:
            today_date = datetime.date.today().isoformat()
            output_data = {
                "Date": today_date,
                "MessageID": context.aws_request_id if context else "test-message-id",
                "Body": response_text,
            }

    except ClientError as e:
        output_data = {
            "Date": datetime.date.today().isoformat(),
            "MessageID": context.aws_request_id if context else "test-message-id",
            "Error": str(e)
        }

    return {
        "statusCode": 200,
        "body": output_data
    }
